# Projeto em Python: **Sistema de Cadastro de usuários**

## **BIBLIOTECAS UTILIZADAS**


1.  **JSON**

  Suporte para serialização e desserialização de arquivos no formato de dados JSON (JavaScript Object Notation).
  
  **Conceito:**

  "JavaScript Object Notation (JSON) é um formato baseado em texto padrão para representar dados estruturados com base na sintaxe do objeto JavaScript. É comumente usado para transmitir dados em aplicativos da Web (por exemplo, enviar alguns dados do servidor para o cliente, para que possam ser exibidos em uma página da Web ou vice-versa)."

  **Documentação de referência:**

  https://docs.python.org/pt-br/3/library/json.html?highlight=import%20json

  https://developer.mozilla.org/pt-BR/docs/Learn/JavaScript/Objects/JSON



  
2.  **CSV**

  Utilizada para trabalhar com arquivos no formato tabular, por meio de linhas e colunas. A padronização da representação de dados CSV ( Comma-Separated Values) foi publicada na RFC 4180 em 2005 pela ISOC (Internet Society) através da IETF (Internet Engineering Task Force), organização responsável pelo estabelecimento de padrões de protocolos da internet e tecnologias envolvidas.

  **Documentação de referência:**

  https://datatracker.ietf.org/doc/html/rfc4180

  https://docs.python.org/pt-br/3/library/csv.html?highlight=import%

3.  **from random import randint**

  Permite utilizar a função ranint() por meio do módulo random que implementa geradores de numeros pseudoaleatórios

  **Documentação de referência:**

  https://docs.python.org/3/library/random.html#random.random

4. **google.colab import drive**

  Permite montar o Google Drive em ambiente de execução.

  **Documentação de referência:**

  https://colab.research.google.com/notebooks/io.ipynb


In [1]:
import json
import csv
from random import randint
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


## **FUNCIONALIDADES**
### lerSalvarArquivo


In [3]:
def lerSalvarArquivo(nomeArquivo=None, ler=True, planilha=None):

    if ler:

        nomeArquivo = input("\nDigite o caminho do arquivo JSON no Google Drive (ex: /content/drive/MyDrive/exemplo.json): ")

        try:
            f = open(nomeArquivo, 'r')
            planilha = json.load(f)
            f.close()

        except FileNotFoundError:
            print(f'✕ Arquivo {nomeArquivo} não encontrado.', end='\n\n')

        except json.JSONDecodeError:
            print(f'✕ Erro ao decodificar o arquivo {nomeArquivo}.', end='\n\n')

        finally:
            return nomeArquivo, planilha

    else:
        with open(nomeArquivo, 'w') as outfile:
            json.dump(planilha, outfile, indent = 4, ensure_ascii = False)

## insertNome

In [4]:
def insertNome(mensagem):

    while True:

        nome = input(mensagem)

        if not nome.replace(' ', '').isalpha() or not nome:
            print('✕ Nome inválido. Tente novamente.')

        else:
            return nome

## insertTelefone

In [5]:
def insertTelefone(mensagem):

    while True:

        telefone = input(mensagem)

        if telefone.isnumeric() or not telefone:
            return telefone

        else:
            print('✕ Telefone inválido. Tente novamente.')

## insertSenha

In [6]:
def insertSenha(mensagem):

    while True:
        print('>> A senha deve conter o mínimo de 8 caracteres (mínimo de 2 números e 1 letra maiúscula).')

        senha = input(mensagem)

        countNum, countM = 0, 0
        for char in senha:
            if char.isupper(): countM += 1
            elif char.isnumeric(): countNum += 1

        if countNum < 2 or countM < 1 or len(senha) < 8:
            print('✕ Senha fraca. Tente novamente.')
        else:
            return senha

## gerarId

In [7]:
def gerarId(nome):

    produto = 1
    for letra in nome:
        produto *= ord(letra)

    id = str(produto % 10000)

    return id

## gerarSenha

In [8]:
def gerarSenha():

    senha = ''.join([chr(randint(33,127)) for i in range(8)])

    return senha

## addUsuarioUnico

In [9]:
def addUsuarioUnico(nome, telefone, endereco, senha, planilha):

    if not nome:
        print('✕ Nome inválido. Não é possível adicionar usuário.')
        return None

    id = gerarId(nome)

    if not telefone: telefone = 'Não informado'

    if not endereco: endereco = 'Não informado'

    if not senha: senha = gerarSenha()

    while True:

        if planilha.get(id):
            if planilha[id]['Status'] == False and planilha[id]['Telefone'] == telefone and planilha[id]['Endereço'] == endereco:
                planilha[id]['Status'] = True
                planilha[id]['Senha'] = senha
                print('✓ A conta do usuário foi reativada.')
                break
            id = str(int(id) + 1)

        else:
            planilha[id] = dict(Status = True, Nome = nome, Telefone = telefone, Endereço = endereco, Senha=senha)
            break

## addUsuarios

In [10]:
def addUsuarios(qtd, planilha):

    for i in range(1, qtd + 1):

        nome = insertNome(f'\nNome do {i}° usuário: ')
        telefone = insertTelefone(f'Telefone do {i}° usuário: ')
        endereco = input(f'Endereço do {i}° usuário: ')
        senha = insertSenha(f'Senha do {i}° usuário: ')

        addUsuarioUnico(nome, telefone, endereco, senha, planilha)

        print(f'\n✓ {i}° usuário adicionado com sucesso.')


## addUsuariosCSV

In [11]:
def addUsuariosCSV(planilha):

    nomeArquivo = input("\nDigite o caminho do arquivo CSV no Google Drive (ex: /content/drive/MyDrive/exemplo.csv): ")

    try:
        with open(nomeArquivo, newline='') as csvfile:
            leitor_csv = csv.DictReader(csvfile)

            if not all(chave.capitalize() in leitor_csv.fieldnames for chave in ['Nome', 'Telefone', 'Endereço','Senha']):
                print("✕ O arquivo CSV não possui os cabeçalhos corretos (Nome, Telefone, Endereço, Senha).")
                return

            for linha in leitor_csv:
                nome = linha.get('Nome')
                telefone = linha.get('Telefone')
                endereco = linha.get('Endereço')
                senha = linha.get('Senha')
                addUsuarioUnico(nome, telefone, endereco, senha, planilha)

            print('\n✓ Usuários adicionados com sucesso.', end='\n\n')

    except FileNotFoundError:
        print("✕ Arquivo não encontrado.")
    except csv.Error:
        print("✕ Erro ao ler o arquivo CSV. Verifique se o arquivo está no formato correto.")

## checkUsuario

In [12]:
def checkUsuario(id, planilha):

    if not planilha.get(id):
        return False

    if planilha[id]['Status'] == False:
        return False

    return True

##  checkSenha

In [13]:
def checkSenha(id, planilha):

    for i in range(3):

        senha = input(f'\nDigite a senha do usuário {id}: ')

        if planilha[id]['Senha'] == senha:
            return True

        if i != 2:
            print(f'\n>> Você tem mais {2 - i} tentativas.')

    print('\n✕ Você esgotou as tentativas. Aguarde um pouco e tente mais tarde.', end='\n\n')

    return False

## excluirUsuario

In [14]:
def excluirUsuario(id, planilha):

    if checkSenha(id, planilha):

        planilha[id]['Status'] = False

        print(f'\n✓ Usuário do ID {id} excluído com sucesso.', end='\n\n')

    else:
        print('✕ Erro na exclusão.', end='\n\n')

## editUsuario

In [15]:
def editUsuario(id, planilha):

    print(f'\nPara o ID {id}, qual informação deseja alterar?')
    print('1 - Nome')
    print('2 - Telefone')
    print('3 - Endereço')
    print('4 - Senha')
    print('5 - Sair/Próximo ID')

    if checkSenha(id, planilha):

        while True:

            userInput = input('\nDigite a opção a ser alterada: ')

            if userInput == '5':
                print('\n>> Você escolheu: 5 - Sair/Próximo ID', end='\n\n')
                break

            elif userInput == '1':
                print('\n>> Você escolheu: 1 - Nome', end='\n\n')

                nome = insertNome('Digite o novo nome do usuário: ')

                planilha[id]['Nome'] = nome

                print('\n✓ Nome atualizado com sucesso.')

            elif userInput == '2':
                print('\n>> Você escolheu: 2 - Telefone', end='\n\n')

                telefone = insertTelefone('Digite o novo telefone do usuário: ')

                planilha[id]['Telefone'] = telefone

                print('\n✓ Telefone atualizado com sucesso.')

            elif userInput == '3':
                print('\nVocê escolheu: 3 - Endereço', end='\n\n')

                endereco = input('Digite o novo endereço do usuário: ')

                planilha[id]['Endereço'] = endereco

                print('\n✓ Endereço atualizado com sucesso.')

            elif userInput == '4':
                print('\nVocê escolheu: 4 - Senha', end='\n\n')

                senha = insertSenha('Digite a nova senha do usuário: ')

                planilha[id]['Senha'] = senha

                print('\n✓ Senha atualizada com sucesso.')

            else:
                print('✕ Esta opção não está disponível. Tente novamente.', end='\n\n')

        print(f'✓ Usuário do ID {id} atualizado com sucesso.', end='\n\n')

    else:
        print('✕ Não foi possível alterar dados do usuário.', end='\n\n')

## recuperarSenha

In [16]:
def recuperarSenha(id, planilha):

    print(f'\n>> Para recuperar a senha do ID {id}, precisamos validar algumas informações.', end='\n\n')

    for i in range(3):

        nome = input('Nome do usuário: ')
        telefone = input('Telefone do usuário: ')
        endereco = input('Endereço do usuário: ')

        if nome == planilha[id]['Nome'] and telefone == planilha[id]['Telefone'] and endereco == planilha[id]['Endereço']:
            return f'>> Sua senha é: {planilha[id]["Senha"]}'

        if i != 2:
            print(f'✕ Informações erradas. Você tem mais {2 - i} tentativas.', end='\n\n')

    return f'✕ Não foi possível recuperar senha.'

## viewUsuario

In [17]:
def viewUsuario(id, planilha):

    if checkSenha(id, planilha):

        print(f'\nID: {id}')

        for key, value in planilha[id].items():
            print(f'{key}: {value}')

        print()

    else:
        print('✕ Não foi possível visualizar dados do usuário.', end='\n\n')

##  viewAllUsuarios

In [18]:
def viewAllUsuarios(planilha):

    if checkSenha('9999', planilha):

        for id in planilha:

            if planilha[id]['Status'] == False:
                continue

            print(f'\nID: {id}')

            for key, value in planilha[id].items():
                print(f'{key}: {value}')

        print()

    else:
        print('✕ Não foi possível visualizar dados de todos os usuários.', end='\n\n')

## exportCSV

In [19]:
def exportCSV(planilha):

    try:
        path = input('Informe o endereço do Drive em que deseja salvar o arquivo (ex: /content/drive/MyDrive): ')
        nomeArquivo = '/' + input('Informe o nome do arquivo: ')+'.csv'

        with open(path+nomeArquivo, 'w', newline = '') as f:
            colunas = ['ID', 'Status', 'Nome', 'Telefone', 'Endereço', 'Senha']
            writer = csv.DictWriter(f, fieldnames=colunas)

            writer.writeheader()
            for id in planilha:
                if checkUsuario(id, planilha):
                    linha = dict(ID=id)
                    linha.update(planilha[id])
                    writer.writerow(linha)

        print('\n✓ Arquivo exportado com sucesso para o Drive.', end='\n\n')

    except Exception as e:
        print(f'✕ Erro no export do arquivo: {e}.')

## menu inicial (main)

In [20]:
def main():

    print('-------------- Sistema de Cadastro de Pessoas --------------',end='\n\n')

    nomeArquivo, cadastros = lerSalvarArquivo()

    while True:

        if not cadastros: break

        print('Escolha uma das ações a seguir:\n')
        print('1 - Inserir usuário')
        print('2 - Inserir usuários em Lote')
        print('3 - Excluir usuário')
        print('4 - Atualizar usuário')
        print('5 - Recuperar senha de usuário')
        print('6 - Informações de um usuário')
        print('7 - Informações de todos os usuários')
        print('8 - Exportar informações de usuários em CSV')
        print('9 - Sair\n')

        userInput = input('Digite o número da ação: ')

        if userInput == '9':
            print('\n>> Você escolheu: 9 - Sair')
            print('>> Até a próxima.')
            break

        elif userInput == '1':
            print('\n>> Você escolheu: 1 - Inserir usuário', end='\n\n')

            nome = insertNome('Nome do usuário: ')
            telefone = insertTelefone('Telefone do usuário: ')
            endereco = input('Endereço do usuário: ')
            senha = insertSenha('Senha do usuário: ')

            addUsuarioUnico(nome, telefone, endereco, senha, planilha=cadastros)

            print('\n✓ Usuário adicionado com sucesso.', end='\n\n')

        elif userInput == '2':
            print('\n>> Você escolheu: 2 - Inserir usuários em Lote', end='\n\n')

            while True:
                userInput = input('>> Deseja inserir por quantidade limitada (digite 1) ou arquivo CSV (digite 2)? ')

                if userInput == '1':

                    qtd = int(input('Quantidade de usuários que deseja inserir: '))

                    addUsuarios(qtd, planilha=cadastros)

                    print(f'\n✓ {qtd} usuários adicionados com sucesso.', end='\n\n')

                    break

                if userInput == '2':

                    addUsuariosCSV(planilha=cadastros)

                    break

                else:
                    print('✕ Opção inválida. Tente novamente.')

        elif userInput == '3':
            print('\n>> Você escolheu: 3 - Excluir usuário', end='\n\n')

            while True:

                userInput = tuple(input('ID do usuário a ser excluído (se for mais de um, separe por espaço): ').split())

                token = True
                for id in userInput:
                    if not checkUsuario(id, planilha=cadastros):
                        print(f'\n✕ Usuário com ID {id} não foi encontrado. Insira o(s) ID(s) novamente.', end='\n\n')
                        token = False
                    else:
                        excluirUsuario(id, planilha=cadastros)

                if token: break

        elif userInput == '4':
            print('\n>> Você escolheu: 4 - Atualizar usuário', end='\n\n')

            while True:

                userInput = tuple(input('ID do usuário a ser atualizado (se for mais de um, separe por espaço): ').split())

                token = True
                for id in userInput:
                    if not checkUsuario(id, planilha=cadastros):
                        print(f'\n✕ Usuário com ID {id} não foi encontrado. Insira o(s) ID(s) novamente.', end='\n\n')
                        token = False
                    else:
                        editUsuario(id, planilha=cadastros)

                if token: break

        elif userInput == '5':
            print('\nVocê escolheu: 5 - Recuperar senha de usuário', end='\n\n')

            while True:

                userInput = tuple(input('ID do usuário para recuperar a senha (se for mais de um, separe por espaço): ').split())

                token = True
                for id in userInput:
                    if not checkUsuario(id, planilha=cadastros):
                        print(f'\n✕ Usuário com ID {id} não foi encontrado. Insira o(s) ID(s) novamente.', end='\n\n')
                        token = False
                    else:
                        result = recuperarSenha(id, planilha=cadastros)
                        print(result, end='\n\n')

                if token: break

        elif userInput == '6':
            print('\nVocê escolheu: 6 - Informações de um usuário', end='\n\n')

            while True:

                userInput = tuple(input('ID do usuário a ser visualizado (se for mais de um, separe por espaço): ').split())

                token = True
                for id in userInput:
                    if not checkUsuario(id, planilha=cadastros):
                        print(f'\n✕ Usuário com ID {id} não foi encontrado. Insira o(s) ID(s) novamente.', end = '\n\n')
                        token = False
                    else:
                        viewUsuario(id, planilha=cadastros)

                if token: break

        elif userInput == '7':
            print('\n>> Você escolheu: 7 - Informações de todos os usuários')

            viewAllUsuarios(planilha=cadastros)

        elif userInput == '8':
            print('\n>> Você escolheu: 8 - Exportar informações de usuários em CSV')

            exportCSV(planilha=cadastros)

        else:
            print('\n✕ Esta não é uma opção válida. Tente novamente.', end='\n\n')

        lerSalvarArquivo(nomeArquivo, ler=False, planilha=cadastros)

In [21]:
main()

-------------- Sistema de Cadastro de Pessoas --------------


Digite o caminho do arquivo JSON no Google Drive (ex: /content/drive/MyDrive/exemplo.json): /content/drive/MyDrive/projetoModuloII.json
Escolha uma das ações a seguir:

1 - Inserir usuário
2 - Inserir usuários em Lote
3 - Excluir usuário
4 - Atualizar usuário
5 - Recuperar senha de usuário
6 - Informações de um usuário
7 - Informações de todos os usuários
8 - Exportar informações de usuários em CSV
9 - Sair

Digite o número da ação: 1

>> Você escolheu: 1 - Inserir usuário

Nome do usuário: Leticia
Telefone do usuário: 8756
Endereço do usuário: Rua Random 
>> A senha deve conter o mínimo de 8 caracteres (mínimo de 2 números e 1 letra maiúscula).
Senha do usuário: 34Testeteste

✓ Usuário adicionado com sucesso.

Escolha uma das ações a seguir:

1 - Inserir usuário
2 - Inserir usuários em Lote
3 - Excluir usuário
4 - Atualizar usuário
5 - Recuperar senha de usuário
6 - Informações de um usuário
7 - Informações de todos os usu

KeyboardInterrupt: ignored